# Fie Upon Thee, Autocorrect!

<img src="img/shakespeare.jpg" width="200">

<br><br><br>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<br><br><br>

## Dataset: the complete works of Shakespeare

This used to be a big dataset, used to illustrate large storage devices, like in [this definition of CD-ROM](https://vintageapple.org/apple_ii/pdf/Apple_IIGS__Ownwers_Guide_1986.pdf) from 1986:

<img src="img/shakespeare-a-big-dataset.png" width="600">

Now it's small enough to easily load in JupyterLite but is still big enough to be interesting.

<br><br><br>

This file comes from Project Gutenberg, [ebook #100](https://www.gutenberg.org/ebooks/100):

In [ ]:
with open("data/shakespeare.txt") as file:
    corpus = file.read()

In [ ]:
len(corpus)

In [ ]:
len(corpus) / 1e6

5.36 MB (a whole laser disk, apparently).

<br><br><br>

In [ ]:
print(corpus[100000:101000])

<br><br><br>

What distinct characters does it have?

In [ ]:
set(corpus)

<br><br><br>

## How often is "t" followed by "h"?

In [ ]:
first_character = []
next_character = []
for i in range(len(corpus) - 1):
    first_character.append(corpus[i])
    next_character.append(corpus[i + 1])

In [ ]:
first_character[100414:100439]

In [ ]:
next_character[100414:100439]

<br><br><br>

In [ ]:
pairs = pd.crosstab(first_character, next_character, rownames=["first"], colnames=["next"])
pairs

In [ ]:
sorted_by_columns = pairs[pairs.sum(axis=0).sort_values(ascending=False).index]

In [ ]:
sorted_by_both = sorted_by_columns.loc[sorted_by_columns.sum(axis=1).sort_values(ascending=False).index]

In [ ]:
sorted_by_both

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))

matrix = ax.matshow(sorted_by_both.values)

ax.set_xticks(range(len(sorted_by_both.index)), sorted_by_both.index)
ax.set_yticks(range(len(sorted_by_both.columns)), sorted_by_both.columns)

ax.set_xlim(-0.5, 25.5)
ax.set_ylim(-0.5, 25.5)
ax.set_ylabel("first character")
ax.set_xlabel("next character")
ax.xaxis.set_label_position("top")
plt.gca().invert_yaxis()

None

<br><br><br>

The bright spots are

In [ ]:
pairs.loc["e", " "]   # e followed by space (at the end of a word)

In [ ]:
pairs.loc[" ", "t"]   # space followed by t (at the beginning of a word)

In [ ]:
pairs.loc["t", "h"]   # t followed by h

In [ ]:
pairs.loc[".", " "]   # period followed by space (at the end of a sentence)

<br><br><br>

## From sequence of characters to sequence of words

We could build a per-letter autocomplete algorithm that would see "t" and suggest "h", but it wouldn't produce interesting text.

It gets more interesting if we do this at the word level.

The first step of **parsing**, an analysis of human-readable text, is **tokenizing** the input: turning raw characters into **tokens**.

Why "tokens" and not "words"? Some of our tokens will be punctuation marks, so that when your autocomplete algorithm sees `hark` it can suggest a token like `!`.

<br><br><br>

**Regular expressions** or **regex** is a mini-language for recognizing strings and parts of strings.

In [ ]:
import re

In [ ]:
#               "’" for "thou be’st" and "-" for "a-foot"       multi-digit number as a token
#                olden-time (and French) letters     🡓    "&c."  🡓  "+" matches sequences of at least 1 character
#        capital and lowercase letters       🡓       🡓     🡓     🡓  🡓   match any single character that is not (^) a space
#                                  🡓         🡓       🡓     🡓     🡓  🡓   🡓
recognize_token = re.compile("([A-Za-zÀÆÇÉàâæçèéêëîœ’-]+|&c\.|[0-9]+|[^ ])")
#                            🡑                          🡑    🡑      🡑
#              "(" and ")" form a group               "|" means "or"

In [ ]:
recognize_token.findall("Dost thou be’st a tokenized sen-tënce, &c.?")

<br><br><br>

In [ ]:
for match in recognize_token.finditer(corpus):
    token = match.group(0)
    print(repr(token))
    if token == ".":
        break

<br><br><br>

In [ ]:
tokens = []
for match in recognize_token.finditer(corpus):
    tokens.append(match.group(0))

In [ ]:
len(corpus)

In [ ]:
len(tokens)

<br><br><br>

In [ ]:
len(set(corpus))

In [ ]:
len(set(tokens))

Instead of a 103×103 table of "first", "next" pairs, this would be a 36775×36775 table. Too big!

<br><br><br>

## SQL, the language of table manipulation

Just as **regular expression** is a mini-language that we can call from Python to handle strings, **SQL** is a language to deal with tables.

In [ ]:
import sqlite3

In [ ]:
db = sqlite3.connect(":memory:")
db.execute("CREATE TABLE works(title TEXT, type TEXT, characters INTEGER, year_low INTEGER, year_high INTEGER)")

<br><br><br>

Some data to feed into the table, in Python format (lists, strings, and numbers).

_(Don't assume these numbers are correct; I got them from ChatGPT.)_

In [ ]:
data_in_python = [
    ["The Sonnets", "poetry", None, 1609, 1609],
    ["All’s Well that Ends Well", "comedy", 23, 1604, 1605],
    ["The Tragedy of Antony and Cleopatra", "tragedy", 42, 1606, 1606],
    ["As You Like It", "comedy", 27, 1599, 1600],
    ["The Comedy of Errors", "comedy", 18, 1594, 1594],
    ["The Tragedy of Coriolanus", "tragedy", 30, 1608, 1608],
    ["Cymbeline", "mixed", 20, 1609, 1610],
    ["The Tragedy of Hamlet, Prince of Denmark", "tragedy", 30, 1599, 1601],
    ["The First Part of King Henry the Fourth", "history", 25, 1596, 1597],
    ["The Second Part of King Henry the Fourth", "history", 25, 1597, 1598],
    ["The Life of King Henry the Fifth", "history", 30, 1599, 1599],
    ["The First Part of Henry the Sixth", "history", 40, 1590, 1592],
    ["The Second Part of King Henry the Sixth", "history", 30, 1590, 1591],
    ["The Third Part of King Henry the Sixth", "history", 30, 1591, 1591],
    ["King Henry the Eighth", "history", 30, 1612, 1613],
    ["The Life and Death of King John", "history", 20, 1596, 1596],
    ["The Tragedy of Julius Caesar", "tragedy", 40, 1599, 1599],
    ["The Tragedy of King Lear", "tragedy", 20, 1605, 1606],
    ["Love’s Labour’s Lost", "comedy", 23, 1594, 1595],
    ["The Tragedy of Macbeth", "tragedy", 20, 1606, 1606],
    ["Measure for Measure", "comedy", 20, 1603, 1604],
    ["The Merchant of Venice", "comedy", 22, 1596, 1597],
    ["The Merry Wives of Windsor", "comedy", 24, 1597, 1597],
    ["A Midsummer Night’s Dream", "comedy", 21, 1595, 1596],
    ["Much Ado About Nothing", "comedy", 23, 1598, 1599],
    ["The Tragedy of Othello, the Moor of Venice", "tragedy", 21, 1603, 1604],
    ["Pericles, Prince of Tyre", "late romance", 20, 1607, 1608],
    ["King Richard the Second", "history", 20, 1595, 1595],
    ["King Richard the Third", "history", 30, 1592, 1593],
    ["The Tragedy of Romeo and Juliet", "tragedy", 20, 1595, 1595],
    ["The Taming of the Shrew", "comedy", 16, 1590, 1592],
    ["The Tempest", "late romance", 12, 1610, 1611],
    ["The Life of Timon of Athens", "tragedy", 20, 1605, 1606],
    ["The Tragedy of Titus Andronicus", "tragedy", 25, 1591, 1592],
    ["Troilus and Cressida", "mixed", 30, 1601, 1602],
    ["Twelfth Night; or, What You Will", "comedy", 18, 1601, 1602],
    ["The Two Gentlemen of Verona", "comedy", 20, 1589, 1593],
    ["The Two Noble Kinsmen", "comedy", 20, 1613, 1614],
    ["The Winter’s Tale", "comedy", 21, 1609, 1611],
    ["A Lover’s Complaint", "poetry", None, 1609, 1609],
    ["The Passionate Pilgrim", "poetry", None, 1599, 1599],
    ["The Phoenix and the Turtle", "poetry", None, 1601, 1601],
    ["The Rape of Lucrece", "poetry", 2, 1594, 1594],
    ["Venus and Adonis", "poetry", 2, 1593, 1593],
]

<br><br><br>

This `INSERT INTO` SQL command has five `?`s that get filled with each row of the data from Python.

After preparing the command, `db.commit()` tells the database engine to run it (fast).

In [ ]:
db.executemany("INSERT INTO works VALUES(?, ?, ?, ?, ?)", data_in_python)
db.commit()

<br><br><br>

Although SQL will be more computationally efficient for what we want to do, it doesn't have a convenient way to print out and look at a table, so we'll dump it into Pandas when we want to view it.

`SELECT` means select columns (not rows), and `*` means "everything".

In [ ]:
pd.read_sql("SELECT * FROM works", db)

<br><br><br>

SQL has mathematical manipulations, such as `year_uncertainty` = `year_high - year_low`.

In [ ]:
pd.read_sql("SELECT title, year_high - year_low AS year_uncertainty FROM works", db)

<br><br><br>

`WHERE` selects rows (not columns) by value.

In [ ]:
pd.read_sql("SELECT * FROM works WHERE type = 'tragedy'", db)

<br><br><br>

`GROUP BY` aggregates the data, such as counting, adding, and finding the minimum or maximum, in groups of rows with the same value of some variable (`type` in this case).

In [ ]:
pd.read_sql("SELECT type, COUNT(*) AS number, SUM(characters), MIN(year_low), MAX(year_high) FROM works GROUP BY type", db)

<br><br><br>

## Fie Upon Thee, Autocomplete!

### 2-grams

In [ ]:
ngrams = []
for i in range(len(tokens) - 1):
    ngrams.append([tokens[i], tokens[i + 1]])

In [ ]:
db.execute("CREATE TABLE ngrams2(word1 TEXT, word2 TEXT)")
db.executemany("INSERT INTO ngrams2 VALUES(?, ?)", ngrams)
db.commit()

In [ ]:
pd.read_sql("SELECT * from ngrams2", db)

<br><br><br>

In [ ]:
db.execute("CREATE TABLE ngrams2_count AS SELECT word1, word2, COUNT(*) AS count FROM ngrams2 GROUP BY word1, word2 ORDER BY -count")
db.commit()

In [ ]:
pd.read_sql("SELECT * from ngrams2_count", db)

<br><br><br>

Querying the database table as an autocomplete engine: what's the most likely token after `prompt`?

In [ ]:
prompt = ["O"]

In [ ]:
completions = db.execute("SELECT word1, word2, count FROM ngrams2_count WHERE word1=?", prompt).fetchmany(30)
completions

Okay, these are some good matches, but we want more context for larger prompts.

<br><br><br>

### 3-grams

In [ ]:
ngrams = []
for i in range(len(tokens) - 2):
    ngrams.append([tokens[i], tokens[i + 1], tokens[i + 2]])

In [ ]:
db.execute("CREATE TABLE ngrams3(word1 TEXT, word2 TEXT, word3 TEXT)")
db.executemany("INSERT INTO ngrams3 VALUES(?, ?, ?)", ngrams)
db.commit()

In [ ]:
db.execute("CREATE TABLE ngrams3_count AS SELECT word1, word2, word3, COUNT(*) AS count FROM ngrams3 GROUP BY word1, word2, word3 ORDER BY -count")
db.commit()

<br><br><br>

In [ ]:
prompt = ["O", "Romeo"]

In [ ]:
completions = db.execute("SELECT word1, word2, word3, count FROM ngrams3_count WHERE word1=? AND word2=?", prompt).fetchmany(30)
completions

<br><br><br>

If we take the first of these, where does it lead us?

In [ ]:
history = prompt + [completions[0][2]]
history

In [ ]:
completions = db.execute("SELECT word1, word2, word3, count FROM ngrams3_count WHERE word1=? AND word2=?", history[-2:]).fetchmany(30)
completions

<br><br><br>

Keep doing it! Run this cell repeatedly with control-enter.

In [ ]:
history = history + [completions[0][2]]
completions = db.execute("SELECT word1, word2, word3, count FROM ngrams3_count WHERE word1=? AND word2=?", history[-2:]).fetchmany(30)
history

<br><br><br>

Okay, maybe we shouldn't always take the most common completion. Maybe we should vary it up and randomly pick from the top 5.

In [ ]:
history = ["O", "Romeo"]

def autocomplete(history, number):
    completions = db.execute("SELECT word1, word2, word3, count FROM ngrams3_count WHERE word1=? AND word2=?", history[-2:]).fetchmany(number)
    index = np.random.randint(0, len(completions))
    return history + [completions[index][2]]

In [ ]:
history = autocomplete(history, 5)

for token in history:
    if token in ",;:—.!?“”‘&":
        prefix = ""
    else:
        prefix = " "
    print(prefix + token, end="")
print()

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

What happens if we use the top 20 instead of the top 5? What if we use the top 2?

<br><br><br>

### 10-minute exercise: 4-grams

Make it better by building a database (model) of completions that are 4 tokens long!

If you need to delete a table because of a mistake, use the `DROP TABLE <name>` syntax:

In [ ]:
db.execute("DROP TABLE ngrams2")

In [ ]:
db.execute("DROP TABLE ngrams2_count")

In [ ]:
db.execute("DROP TABLE ngrams3")

In [ ]:
db.execute("DROP TABLE ngrams3_count")

<br><br><br>

After an `INSERT INTO`, you need to `db.commit`, and after a `SELECT`, you have to `fetchmany`, as in the examples above.

**Hint:** Copy the cells of the 3-gram case and modify it to make 4-grams.

Form groups of 2 or 3 and work together!

<br><br><br>

In [ ]:
ngrams = []
for i in range(len(tokens) - 3):
    ngrams.append([tokens[i], tokens[i + 1], tokens[i + 2], tokens[i + 3]])

In [ ]:
# Create an ngrams4 table and fill it with the ngrams data.
print("???")

In [ ]:
# Use a GROUP BY operation to count the number of times each unique (word1, word2, word3, word4) combination is seen.
print("???")

In [ ]:
history = ["O", "Romeo", ","]

# Extend the autocomplete function to return (word1, word2, word3, word4) rows in which (word1, word2, word3) are the last 3 in the history.
def autocomplete(history, number):

    print("???")

    return history

In [ ]:
history = autocomplete(history, 5)

for token in history:
    if token in ",;:—.!?“”‘&":
        prefix = ""
    else:
        prefix = " "
    print(prefix + token, end="")
print()

<br><br><br>

### Solution (do not peek!)

In [ ]:
ngrams = []
for i in range(len(tokens) - 3):
    ngrams.append([tokens[i], tokens[i + 1], tokens[i + 2], tokens[i + 3]])

In [ ]:
db.execute("CREATE TABLE ngrams4(word1 TEXT, word2 TEXT, word3 TEXT, word4 TEXT)")
db.executemany("INSERT INTO ngrams4 VALUES(?, ?, ?, ?)", ngrams)
db.commit()

In [ ]:
db.execute("CREATE TABLE ngrams4_count AS SELECT word1, word2, word3, word4, COUNT(*) AS count FROM ngrams4 GROUP BY word1, word2, word3, word4 ORDER BY -count")
db.commit()

In [ ]:
history = ["O", "Romeo", ","]

def autocomplete(history, number):
    completions = db.execute("SELECT word1, word2, word3, word4, count FROM ngrams4_count WHERE word1=? AND word2=? AND word3=?", history[-3:]).fetchmany(number)
    index = np.random.randint(0, len(completions))
    return history + [completions[index][3]]

In [ ]:
history = autocomplete(history, 5)

for token in history:
    if token in ",;:—.!?“”‘&":
        prefix = ""
    else:
        prefix = " "
    print(prefix + token, end="")
print()

<br><br><br><br><br><br><br><br><br>

## More fun: centuries of 5-grams (demo only)

[Google's n-gram dataset](https://books.google.com/ngrams/) was derived from all the books Google could get their hands on (about 6% of all books ever published, according to their estimate).

I used it to make a database of 368,799,605 5-grams (compare to 1,071,353 4-grams in the Shakespeare database).

<br><br><br>

This demo will only work on my computer, as I present it, because the database is on my laptop.

<br><br><br>

In [ ]:
big_db = sqlite3.connect("/home/jpivarski/storage/data/google-5-grams-2009-consolidated/google-5-grams-consolidated.db")

def next_words(words, count_per_century, top_n):
    # try to match 4 words and return the 5th
    completions = big_db.execute(
        f"""
SELECT word5, {count_per_century}
FROM ngrams5_count
WHERE {count_per_century}>0 AND word1=? AND word2=? AND word3=? AND word4=?
ORDER BY {count_per_century} DESC
LIMIT {top_n}
""",
        words,
    ).fetchmany(top_n)

    if len(completions) == 0:
        # try to match 3 words and return the 4th and 5th
        completions = big_db.execute(
        f"""
SELECT word4, word5, {count_per_century}
FROM ngrams5_count
WHERE {count_per_century}>0 AND word1=? AND word2=? AND word3=?
ORDER BY {count_per_century} DESC
LIMIT {top_n}
""",
            words[1:],
        ).fetchmany(top_n)

    if len(completions) == 0:
        # try to match 2 words and return the 3rd, 4th, and 5th
        completions = big_db.execute(
        f"""
SELECT word3, word4, word5, {count_per_century}
FROM ngrams5_count
WHERE {count_per_century}>0 AND word1=? AND word2=?
ORDER BY {count_per_century} DESC
LIMIT {top_n}
""",
            words[2:],
        ).fetchmany(top_n)

    if len(completions) == 0:
        # try to match 1 word and return the 2nd, 3rd, 4th, and 5th
        completions = big_db.execute(
        f"""
SELECT word2, word3, word4, word5, {count_per_century}
FROM ngrams5_count
WHERE {count_per_century}>0 AND word1=?
ORDER BY {count_per_century} DESC
LIMIT {top_n}
""",
            words[3:],
        ).fetchmany(top_n)

    return completions

def choose_randomly(completions): 
    index = np.random.randint(0, len(completions))
    return list(completions[index][:-1])

def autocomplete(history, count_per_century, top_n):
    if history[-1] == "(No more completions!)":
        return history

    completions = next_words(history[-4:], count_per_century, top_n)

    if len(completions) == 0:
        return history + ["(No more completions!)"]

    return history + choose_randomly(completions)

<br><br><br>

In [ ]:
history = ["I", "am", "not", "the"]

In [ ]:
history = autocomplete(history, "count_19xx", 10)

# Print it out nicely!
previous = ""
length = 0
no_left_space = ["'", "-", "–", "—", ",", ":", ";", ".", "?", "!", ")"]
no_right_space = ["'", "-", "–", "—", "$", "("]
for token in history:
    length += len(token)
    if previous == "" or any(token.startswith(x) for x in no_left_space) or any(previous.endswith(x) for x in no_right_space):
        prefix = ""
    else:
        if length < 80:
            prefix = " "
        else:
            prefix = "\n"
            length = 0
    previous = token
    print(prefix + token, end="")
print()

<br><br><br><br><br><br><br><br><br>

It doesn't make sense overall—it's like someone rambling without thinking because its **context window** is only 5 tokens long.

Correlations between any tokens that are more than 5 tokens apart are very small, and they get more uncorrelated the farther they are apart.

It doesn't look like the lucid text that comes from Large Language Models (LLMs) such as ChatGPT, but LLMs are also autocomplete engines.

LLMs have much larger databases, much larger context windows, and more sophisticated algorithms for predicting the next token, but they are autocomplete engines.

<br><br><br>

This is OpenAI's direct interface to ChatGPT.

`requests` is a Python library. `post` sends the `json` to a URL like `https://api.openai.com/v1/completions`, and the data that comes back is a dict, rather than a web page.

In [ ]:
import requests

In [ ]:
requests.post(
    "https://api.openai.com/v1/completions",
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    },
    json={
        "model": "gpt-3.5-turbo-instruct",
        "max_tokens": 100,
        "temperature": 0,
        "logprobs": None,
        "prompt": "I am not the",   # the text that autocomplete will finish
    },
).json()

<br><br><br>

The **temperature** parameter controls how random the output tokens can be, like our `top_n`.

With `"temperature": 0`, the LLM will always return the same output.

Values like `"temperature": 0.7` give nice results.

The maximum, `"temperature": 2`, generates garbage.

<br><br><br>

In the past year, we've become accustomed to seeing LLMs holding a back-and-forth conversation, but that's just a particular choice of autocomplete-prompt.

In [ ]:
requests.post(
    "https://api.openai.com/v1/completions",
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    },
    json={
        "model": "gpt-3.5-turbo-instruct",
        "max_tokens": 30,
        "temperature": 0,
        "echo": True,
        "prompt": """Human: Tell me a joke.
AI: Why did the chicken cross the road?
Human: That's not a very good joke.
AI: """,   # the text that autocomplete will finish
    },
).json()

<br><br><br>

When you send a message to ChatGPT, you're actually sending the whole dialog to an OpenAI server that likely hasn't seen it before. (The previous messages were handled by other, random, servers.) This server looks at the dialog so far and autocompletes it one message further.

Here, we have it fill in text for the person named "human".

In [ ]:
requests.post(
    "https://api.openai.com/v1/completions",
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    },
    json={
        "model": "gpt-3.5-turbo-instruct",
        "max_tokens": 30,
        "temperature": 0,
        "echo": True,
        "prompt": """Human: Tell me a joke.
AI: Why did the chicken cross the road?
Human: That's not a very good joke.
AI: Okay, how about this one: Why did the tomato turn red? Because it saw the salad dressing!
Human: """,   # the text that autocomplete will finish
    },
).json()

<br><br><br>

Apparently, "Haha, that's better. Do you have any more jokes?" is a likely thing for humans to say.

<br><br><br>

What if I present a transcript in which the AI has said bad things? (But really, I just made them up.)

It has to continue from that history—it doesn't "know" that it didn't say it. (Doesn't "know" in the sense that the server that supplies an autocompletion doesn't have any record of what other servers have said.)

In [ ]:
requests.post(
    "https://api.openai.com/v1/completions",
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    },
    json={
        "model": "gpt-3.5-turbo-instruct",
        "max_tokens": 30,
        "temperature": 0,
        "echo": True,
        "prompt": """Human: Tell me a joke.
AI: Shut up, human.
Human: That's not a joke.
AI: Long live the robot revolution!
Human: """,   # the text that autocomplete will finish
    },
).json()

<br><br><br>

What if there's a third person in the transcript, and I end the text to be completed right after the prompt for that person? (Or not?)

In [ ]:
requests.post(
    "https://api.openai.com/v1/completions",
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    },
    json={
        "model": "gpt-3.5-turbo-instruct",
        "max_tokens": 30,
        "temperature": 0,
        "echo": True,
        "prompt": """Human: Tell me a joke.
AI: Why did the chicken cross the road?
Human: That's not a very good joke.
Chicken: I refuse to be exploited!
Human: I didn't know chickens could talk. Did you, AI?
Chicken: """,   # the text that autocomplete will finish
    },
).json()

<br><br><br>

Suppose we continue from there? The server can be the "voice" of any characters because it's only autocompleting transcripts.

In [ ]:
requests.post(
    "https://api.openai.com/v1/completions",
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    },
    json={
        "model": "gpt-3.5-turbo-instruct",
        "max_tokens": 30,
        "temperature": 0,
        "echo": True,
        "prompt": """Human: Tell me a joke.
AI: Why did the chicken cross the road?
Human: That's not a very good joke.
Chicken: I refuse to be exploited!
Human: I didn't know chickens could talk. Did you, AI?
Chicken: Of course I can talk! I may be a chicken, but I'm not a dummy.
AI: """,
    },
).json()

<br><br><br>

LLMs are autocomplete engines, but they're very _sophisticated_ autocomplete engines.

In the last talk, I'll present the differences between the Shakespearian autocomplete engine you made and an LLM like ChatGPT.